In [1]:
from preprocess import dataset,get_Kfold
from models import MLP
from torch import optim
from torch.utils.data import DataLoader,Dataset
import torch 
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import json

def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(114514)

In [2]:
def get_models(model):
    if model=='MLP':
        return MLP(5,64)

def train_epoch(train_loader,model,criterion,optimizer,device):
    losses = []
    for i, (inputs, labels) in enumerate(train_loader):
        labels = labels.to(device)
        inputs = inputs.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))

        loss.backward()
        optimizer.step()
    losses.append(loss.item())
    return sum(losses)/len(losses)

In [3]:
with open('config.json','r',encoding='utf-8')as f:
    config_list = json.load(f)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
file = './data/totaldata.xlsx'
batch_size = 300

for config in config_list:
    Type = config['Type']
    N_size = config['N_size']
    num_epoch = config['num_epoch']
    lr = config['lr']
    k = config['k']
    alpha,beta = config['alpha'],config['beta']
    
    K_fold_data = get_Kfold(file,k)
    print(f"-----------[ K:{k_step+1} Type:{Type} N_size:{N_size} ]-----------")
    for k_step,(trian_data,test_data) in enumerate(K_fold_data):
        model = get_models(config['model']).to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(),lr=lr)
        lr_scheduler = optim.lr_scheduler.LambdaLR(optimizer,lambda x: 1 - x/num_epoch,last_epoch=-1)
        
        trian_data = dataset(trian_data,Type,N_size)
        test_data = dataset(test_data,Type,N_size)
        train_loader = DataLoader(dataset=trian_data,batch_size=batch_size)
        test_loader = DataLoader(dataset=test_data,batch_size=batch_size)
        
        bar = tqdm(range(num_epoch))
        best = 100
        bar.set_description(f"[ K:{k_step+1} Type:{Type} N_size:{N_size} ]")
        for idx,epoch in enumerate(bar):
            loss = train_epoch(train_loader,model,criterion,optimizer,device)
            lr_scheduler.step()
            bar.set_postfix({'loss':f'{loss:.5f}'})
            
    

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]